In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', balance=True, dataset_folder=dataset_folder)

In [20]:
dataset.ori_test[0]

array(['40.27378464', 'White', 'Female', '7', '132', '0.927777767',
       'Chest Pain', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No'],
      dtype=object)

In [10]:
dataset.train.shape

(6688, 14)

In [19]:
dataset.train[:,12] = 1
msk = np.where(dataset.labels_train == 1)[0]
msk = msk[:int(msk.shape[0]*0.85)]
dataset.train[msk,12] = 0

dataset.test[:,12] = 1
dataset.ori_test[mskk,12] = 'Yes'
msk = np.where(dataset.labels_test == 1)[0]
print(msk.shape)
mskk = msk[:int(msk.shape[0]*0.85)]
print(mskk.shape)
dataset.test[mskk,12] = 0
dataset.ori_test[mskk,12] = 'No'

(436,)
(370,)


In [13]:
explainer = anchor_tabular.AnchorTabularExplainer(dataset.class_names, dataset.feature_names, dataset.data, dataset.categorical_names)
explainer.fit(dataset.train, dataset.labels_train, dataset.validation, dataset.labels_validation)

In [14]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state= 46)
c.fit(explainer.encoder.transform(dataset.train), dataset.labels_train)
predict_fn = lambda x: c.predict(explainer.encoder.transform(x))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))
print('Train', sklearn.metrics.roc_auc_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.roc_auc_score(dataset.labels_test, predict_fn(dataset.test)))

Train 0.9910287081339713
Test 0.937799043062201
Train 0.9910114519944965
Test 0.9393348623853212


In [6]:
exp_fn_bulk = lambda x: np.array([np.array(explainer.explain_instance(x[idx], c.predict, threshold=0.95).features()) for idx in range(len(x))])

In [7]:
exp1 = exp_fn_bulk(dataset.test[:3])

In [8]:
exp1

array([[12],
       [12],
       [12]])

In [9]:
num_feats = len(dataset.feature_names)
imp = np.zeros(num_feats).astype(int)

In [10]:
for i in range(len(dataset.test)):
    exp = explainer.explain_instance(dataset.test[i], c.predict, threshold=0.95).features()
    for j in exp:
        imp[j] = imp[j] + 1

In [16]:
imp

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 836,
         0])

In [17]:
import copy
import pandas as pd
feats = copy.deepcopy(dataset.feature_names)
feats.append('death')
ori_data = pd.DataFrame(dataset.ori_test, columns=feats)

In [18]:
order = np.argsort(-imp)
a = list(np.array(feats)[order])
a

['smoke',
 'age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'afib2']

In [21]:
z = ['death']
z.extend(a)

In [27]:
z

['death',
 'smoke',
 'age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'afib2']

In [28]:
view1 = ori_data.reindex(columns=z)

In [29]:
preds =  predict_fn(dataset.test)

In [31]:
view1.insert(loc=1,column='prediction',value=preds)

In [33]:
yesmap={0:'No', 1:'Yes'}
yes_no = lambda x: yesmap[x] 
rm_comma = lambda x: x.replace(',','')

In [34]:
view1['prediction']=view1['prediction'].apply(yes_no)
view1['reason']=view1['reason'].apply(rm_comma)

In [ ]:
view1

In [36]:
view1.to_csv('bmortanchor.csv',index=False)